In [45]:
! pip install lightbgm
#!conda install scikit-learn
import os
import tensorflow as tf	
import numpy as np
import time
from tf_utils import input_fn_from_dataset,input_fn_frame_from_dataset,save_tf_record,prob_positive_class_from_prediction
from get_data import get_videos_from_folder,get_target_from_csv
from utils import save_solution
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
                                        MaxPooling2D)
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from keras import backend as K
from keras import metrics
from sklearn.utils.validation import check_array
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

  Could not find a version that satisfies the requirement lightbgm (from versions: )
No matching distribution found for lightbgm


In [7]:
pframe=40

In [32]:
train_folder = os.path.join("../train/")
test_folder = os.path.join("../test/")

train_target = os.path.join('../train_target.csv')
my_solution_file = os.path.join('../solution.csv')

tf_record_dir = os.path.join('..','tf_records')
os.makedirs(tf_record_dir, exist_ok=True)

tf_record_train = os.path.join(tf_record_dir, 'train' + '.tfrecords')
tf_record_test = os.path.join(tf_record_dir, 'test' + '.tfrecords')
x_train = get_videos_from_folder(train_folder)
y_train = get_target_from_csv(train_target)
X = np.zeros((x_train.shape[0], pframe, 100, 100,1))

for i in range(x_train.shape[0]):
    if x_train[i].shape[0] >= pframe:
        X[i,: pframe,:x_train[i].shape[1],:x_train[i].shape[2],0] = x_train[i][: pframe,:x_train[i].shape[1],:x_train[i].shape[2]]
    else:
        # repeated padding
        frames, height, width = x_train[i].shape[0], x_train[i].shape[1], x_train[i].shape[2]
        pos = 0
        while pos + frames <= pframe:
            X[i,pos:(pos+frames),:height,:width,0] = x_train[i][:frames,:height,:width]
            pos += frames
        if pos < pframe:
            X[i,pos:pframe,:height,:width,0] = x_train[i][:(pframe-pos),:height,:width]

In [41]:
y_train = get_target_from_csv(train_target)

In [ ]:
def cross_validate( split_size=5):
    results = []
    kf = KFold(n_splits=split_size)
    for train_idx, val_idx in kf.split(X, y_train):
        train_x = X[train_idx]
        train_y = y_train[train_idx]
        val_x = X[val_idx]
        val_y = y_train[val_idx]
        His=model.fit(train_x, train_y, batch_size=32, validation_data=(val_x,val_y),verbose=1,callbacks=[tb, early_stopper, csv_logger], epochs=nb_epoch)
        results.append(His,history)
    return results
results=cross_validate(5)

In [5]:
x_test = get_videos_from_folder(test_folder)

In [9]:
X_test = np.zeros((x_test.shape[0],  pframe, 100, 100,1))
for i in range(x_test.shape[0]):
    if x_test[i].shape[0]>= pframe:
        X_test[i,: pframe,:x_test[i].shape[1],:x_test[i].shape[2],0] = x_test[i][: pframe,:x_test[i].shape[1],:x_test[i].shape[2]]
    else:
        # repeated padding
        frames, height, width = x_test[i].shape[0], x_test[i].shape[1], x_test[i].shape[2]
        pos = 0
        while pos + frames <= pframe:
            X_test[i,pos:(pos+frames),:height,:width,0] = x_test[i][:frames,:height,:width]
            pos += frames
        if pos < pframe:
            X_test[i,pos:pframe,:height,:width,0] = x_test[i][:(pframe-pos),:height,:width]

In [10]:
x_cell_edges = np.linspace(0,40,4,dtype=int)
y_cell_edges = np.linspace(0,100,9,dtype=int)
z_cell_edges = np.linspace(0,100,9,dtype=int)

        # histograms
histogram_train = np.zeros((X.shape[0], 3,8,8, 45))
histogram_test = np.zeros((X_test.shape[0], 3,8,8, 45))

In [11]:
for i in range(0, X.shape[0]):
    image_3D = X[i, :, :, :,0]

    for xi in range(0, x_cell_edges.size - 1):
        for yi in range(0, y_cell_edges.size - 1):
            for zi in range(0, z_cell_edges.size - 1):

                        # image block for histogram
                image_block = image_3D[
                                      x_cell_edges[xi]:x_cell_edges[xi+1],
                                      y_cell_edges[yi]:y_cell_edges[yi+1],
                                      z_cell_edges[zi]:z_cell_edges[zi+1]]

                        # histogram
                histogram_train[i, xi, yi, zi, :], bins = \
                            np.histogram(image_block,
                                         bins=np.linspace(0,
                                                          4500,
                                                          45 + 1))

X_new = np.reshape(histogram_train, (X.shape[0], -1))


In [48]:
for i in range(0, X_test.shape[0]):
    image_3D = X_test[i, :, :, :,0]

    for xi in range(0, x_cell_edges.size - 1):
        for yi in range(0, y_cell_edges.size - 1):
            for zi in range(0, z_cell_edges.size - 1):

                        # image block for histogram
                image_block = image_3D[
                                      x_cell_edges[xi]:x_cell_edges[xi+1],
                                      y_cell_edges[yi]:y_cell_edges[yi+1],
                                      z_cell_edges[zi]:z_cell_edges[zi+1]]

                        # histogram
                histogram_test[i, xi, yi, zi, :], bins = \
                            np.histogram(image_block,
                                         bins=np.linspace(0,
                                                          4500,
                                                          45 + 1))

X_test_new = np.reshape(histogram_test, (X_test.shape[0], -1))

In [42]:
X_train, X_val, y_train, y_val = train_test_split(X_new, y_train, test_size = 0.2)

In [43]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

In [46]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

def custom_r2(preds, train_data):
    labels = train_data.get_label()
    return 'r2', roc_auc_score(labels, preds), True

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                feval=custom_r2,
                valid_sets={lgb_train, lgb_eval},
                early_stopping_rounds=20)

Start training...
[1]	training's binary_logloss: 0.684001	training's r2: 0.715474	valid_0's binary_logloss: 0.693779	valid_0's r2: 0.539216
Training until validation scores don't improve for 20 rounds.
[2]	training's binary_logloss: 0.6743	training's r2: 0.743448	valid_0's binary_logloss: 0.695332	valid_0's r2: 0.541176
[3]	training's binary_logloss: 0.667527	training's r2: 0.751638	valid_0's binary_logloss: 0.692202	valid_0's r2: 0.562745
[4]	training's binary_logloss: 0.658272	training's r2: 0.797883	valid_0's binary_logloss: 0.687966	valid_0's r2: 0.592157
[5]	training's binary_logloss: 0.649823	training's r2: 0.826235	valid_0's binary_logloss: 0.687125	valid_0's r2: 0.582353
[6]	training's binary_logloss: 0.63848	training's r2: 0.863911	valid_0's binary_logloss: 0.685909	valid_0's r2: 0.57451
[7]	training's binary_logloss: 0.628578	training's r2: 0.86807	valid_0's binary_logloss: 0.687486	valid_0's r2: 0.55098
[8]	training's binary_logloss: 0.618834	training's r2: 0.87752	valid_0's

In [49]:
y_pred = gbm.predict(X_test_new , num_iteration=gbm.best_iteration)

In [50]:
y_pred

array([0.18252207, 0.18251272, 0.67538375, 0.71109725, 0.46787806,
       0.31379273, 0.53665773, 0.81222736, 0.58728616, 0.44650795,
       0.50558975, 0.36421496, 0.3337267 , 0.20201454, 0.47373049,
       0.7770669 , 0.2626228 , 0.75409881, 0.48126645, 0.40506914,
       0.37401944, 0.80731974, 0.38513308, 0.28721722, 0.53106075,
       0.69378906, 0.37366828, 0.78667163, 0.5895963 , 0.17025691,
       0.49923208, 0.54503384, 0.22028171, 0.32793726, 0.54588198,
       0.60984379, 0.32898669, 0.3872737 , 0.63271649, 0.45602464,
       0.35331358, 0.19771535, 0.44437753, 0.83911529, 0.35253823,
       0.69483722, 0.26422159, 0.27896455, 0.81821767, 0.44600597,
       0.6736207 , 0.46303972, 0.47458599, 0.27320103, 0.60779256,
       0.22101977, 0.44619288, 0.77669503, 0.66139745, 0.34131124,
       0.16926377, 0.29520012, 0.68164659, 0.4088777 , 0.76720655,
       0.50909171, 0.56601598, 0.48724602, 0.40484814])

In [51]:
f = open("submission.csv", "w")
f.write("id,y\n")
for i,x in enumerate(y_pred):
    f.write("{},{}\n".format(i,x))
f.close()